In [6]:
import numpy as np 
import pandas as pd 
import json
!cd C:/Users/gbao/Google 云端硬盘/BUDT758B/

In [11]:
import gc

path = 'C:/Users/gbao/Google 云端硬盘/BUDT758B/DATA/'
train_path = 'simplified-nq-train.jsonl'
test_path = 'simplified-nq-test.jsonl'
sample_submission_path = 'sample_submission.csv'

def read_data(path, sample = True, chunksize = 1000):
    if sample == True:
        df = []
        with open(path, 'rt') as reader:
            for i in range(chunksize):
                df.append(json.loads(reader.readline()))
        df = pd.DataFrame(df)
        print('Our sampled dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
    else:
        df = pd.read_json(path, orient = 'records', lines = True)
        print('Our dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
        gc.collect()
    return df

train = read_data(path+train_path, sample = True)
test = pd.read_json(path+test_path, orient = 'records', lines = True)
train.head()

Our sampled dataset have 1000 rows and 6 columns


,document_text,long_answer_candidates,question_text,annotations,document_url,example_id
0,Email marketing - Wikipedia <H1> Email marketi...,"[{'start_token': 14, 'top_level': True, 'end_t...",which is the most common use of opt-in e-mail ...,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Em...,5655493461695504401
1,The Mother ( How I Met Your Mother ) - wikiped...,"[{'start_token': 28, 'top_level': True, 'end_t...",how i.met your mother who is the mother,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Th...,5328212470870865242
2,Human fertilization - wikipedia <H1> Human fer...,"[{'start_token': 14, 'top_level': True, 'end_t...",what type of fertilisation takes place in humans,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Hu...,4435104480114867852
3,List of National Football League career quarte...,"[{'start_token': 28, 'top_level': True, 'end_t...",who had the most wins in the nfl,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Li...,5289242154789678439
4,Roanoke Colony - wikipedia <H1> Roanoke Colony...,"[{'start_token': 32, 'top_level': True, 'end_t...",what happened to the lost settlement of roanoke,"[{'yes_no_answer': 'NONE', 'long_answer': {'st...",https://en.wikipedia.org//w/index.php?title=Ro...,5489863933082811018


In [12]:
def extract_target_variable(df):
    long_answer = []
    for i in range(len(df)):
        long = df['annotations'][i][0]['long_answer']
        if long['start_token'] == -1:
            long_answer.append('EMPTY')
        else:
            st = long['start_token']
            et = long['end_token']
            long_answer.append(f'{st}'+':'+f'{et}')
    long_answer = pd.DataFrame({'long_answer': long_answer})
    return long_answer
        
short_answer = extract_target_variable(train)
short_answer.head()

def build_train_test_long(df, train = True):
    final_long_answer_frame = pd.DataFrame()
    if train == True:
        # get long answer
        long_answer = extract_target_variable(df)
        
        # iterate over each row to get the possible answers
        for index, row in df.iterrows():
            start_end_tokens = []
            questions = []
            documents = []
            responds = []
            IDs = []
            for i in row['long_answer_candidates']:
                start_token = i['start_token']
                end_token = i['end_token']
                start_end_token = str(i['start_token']) + ':' + str(i['end_token'])
                question = row['question_text']
                document = row['document_text']
                ID = row['example_id']
                respond = " ".join(row['document_text'].split()[start_token : end_token])
                start_end_tokens.append(start_end_token)
                questions.append(question)
                documents.append(document)
                responds.append(respond)
                IDs.append(ID)

            long_answer_frame = pd.DataFrame({'ID':IDs, 'document': documents,'question': questions,'respond': responds, 'start_end_token': start_end_tokens})
            long_answer_frame['answer'] = long_answer.iloc[index][0]
            long_answer_frame['target'] = long_answer_frame['start_end_token'] == long_answer_frame['answer']
            long_answer_frame['target'] = long_answer_frame['target'].astype('int16')
            long_answer_frame.drop(['answer'], inplace = True, axis = 1)
            final_long_answer_frame = pd.concat([final_long_answer_frame, long_answer_frame])
        return final_long_answer_frame
    else:
         # iterate over each row to get the possible answers
        for index, row in df.iterrows():
            start_end_tokens = []
            questions = []
            documents = []
            responds = []
            for i in row['long_answer_candidates']:
                start_token = i['start_token']
                end_token = i['end_token']
                start_end_token = str(i['start_token']) + ':' + str(i['end_token'])
                question = row['question_text']
                document = row['doument_text']
                ID = row['example_id']
                respond = " ".join(row['document_text'].split()[start_token : end_token])
                start_end_tokens.append(start_end_token)
                questions.append(question)
                documents.append(documents)
                responds.append(respond)
                IDs.append(ID)

            long_answer_frame = pd.DataFrame({'ID':IDs, 'document': documents,'question': questions,'respond': responds, 'start_end_token': start_end_tokens})
            final_long_answer_frame = pd.concat([final_long_answer_frame, long_answer_frame])
        return final_long_answer_frame

In [13]:

la = build_train_test_long(train)
la = la[la.target == 1]
la['start_token'] = la.start_end_token.apply(lambda x: int(x.split(':')[0]))
la = la[['ID','document','question','respond','start_token']]
la.reset_index(inplace=True,drop=True)

In [14]:
import json
data = np.array(la)

output = {}
output['version'] = 'v1.0'
output['data'] = []

for line in data:
    answers = [] 
    qas = []
    paragraphs = []
    qid = line[0]
    context = line[1]
    question = line[2]
    answer = line[3]
    answer_start = line[-1]
    answers.append({'answer_start': answer_start, 'text': answer})
    qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
    paragraphs.append({'context': context, 'qas': qas})
    output['data'].append({'title': 'None', 'paragraphs': paragraphs})
    
with open('data/QA_train.json', 'w') as outfile:
    json.dump(output,outfile)